In [ ]:
!wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
!bash ./py310.sh -b -f -p /usr/local
!python -m ipykernel install --name "py310" --user

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
print('version',sys.version)

In [ ]:
!pip install networkx[default]
!pip install torch
!pip install lxml
!pip install numpy

In [ ]:
import json
import numpy as np
import networkx as nx
import copy
import matplotlib.pyplot as plt
from xml.dom.minidom import parse, Document
import os
from matplotlib.pyplot import MultipleLocator
import torch
import torch.nn as nn
import torch.nn.functional as F


def get_text(node):
    rc = []
    for node in node.childNodes:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
    return "".join(rc).strip()


def bpmn_to_graph(root_dom: Document, prefix: str = "bpmn"):
    task_nodes = root_dom.getElementsByTagName(f"{prefix}task")
    seq_flows = root_dom.getElementsByTagName(f"{prefix}sequenceFlow")
    par_gateways = root_dom.getElementsByTagName(f"{prefix}parallelGateway")
    exc_gateways = root_dom.getElementsByTagName(f"{prefix}exclusiveGateway")

    all_tasks = task_nodes
    name_nodes_set = {}
    sorted_task_nodes = []
    sorted_name = []

    for i in range(len(all_tasks)):
      task_name = all_tasks[i].getAttribute("name")
      name_nodes_set[task_name] = all_tasks[i]

    for j in range(len(all_tasks)):
      name = all_tasks[j].getAttribute("name")
      sorted_name.append(name)
    sorted_name = sorted(sorted_name, key=str.lower) #返回字母顺序排列的列表

    for k in range(len(all_tasks)):
      name_key = sorted_name[k]
      nodes_value = name_nodes_set.get(name_key)
      sorted_task_nodes.append(nodes_value)

    node_list =  sorted_task_nodes + par_gateways + exc_gateways


    seq_flow_ids = []
    par_gateway_ids = []
    exc_gateway_ids = []

    G = nx.DiGraph()
    for node in node_list + seq_flows:
        node_type = node.tagName
        node_id = node.getAttribute("id")
        node_name = node.getAttribute("name")

        if node_type == f"{prefix}sequenceFlow":
            seq_flow_ids.append(node_id)
        elif node_type == f"{prefix}parallelGateway":
            par_gateway_ids.append(node_id)
        elif node_type == f"{prefix}exclusiveGateway":
            exc_gateway_ids.append(node_id)

        G.add_node(node_id, type=node_type, name=node_name)

    for node in node_list:
        node_id = node.getAttribute("id")

        # for child in node.childNodes:
        #     if child.nodeType == child.TEXT_NODE:
        #         continue
        #     assert child.tagName in [f"{prefix}incoming", f"{prefix}outgoing", "extensionElements"]
        #     child_id = get_text(child)
        #     if child.tagName == f"{prefix}incoming":
        #         G.add_edge(child_id, node_id)
        #     elif child.tagName == f"{prefix}outgoing":
        #         G.add_edge(node_id, child_id)
        for child in node.childNodes:
            if child.nodeType == child.TEXT_NODE:
                continue
            if child.tagName not in [f"{prefix}incoming", f"{prefix}outgoing",
                "extensionElements", f"{prefix}dataOutputAssociation",
                f"{prefix}standardLoopCharacteristics", f"{prefix}multiInstanceLoopCharacteristics",
                 f"{prefix}property", f"{prefix}dataInputAssociation"]:
                # 在这里进行相应的操作或输出错误信息
                print("Invalid tag name:", child.tagName)
            child_id = get_text(child)
            if child.tagName == f"{prefix}incoming":
                G.add_edge(child_id, node_id)
            elif child.tagName == f"{prefix}outgoing":
                G.add_edge(node_id, child_id)


    for node_id in seq_flow_ids + exc_gateway_ids:
        for predecessor in G.predecessors(node_id):
            for successor in G.successors(node_id):
                G.add_edge(predecessor, successor)

        G.remove_node(node_id)

    for node_id in par_gateway_ids:
        for predecessor in G.predecessors(node_id):
            for successor in G.successors(node_id):
                G.add_edge(predecessor, successor, is_par=True)

        G.remove_node(node_id)

    H = copy.deepcopy(G)

    while not nx.is_directed_acyclic_graph(G):
        cycle = nx.find_cycle(G, orientation="original")
        G.remove_edge(*cycle[-1][:2])

    for (u, v), lca in nx.all_pairs_lowest_common_ancestor(G):
        if u == lca or v == lca:
            continue

        ok_u = False
        for path in nx.all_simple_paths(G, lca, u):
            if len(path) > 0:
                attrs = G.get_edge_data(path[0], path[1])
                if "is_par" in attrs and attrs["is_par"]:
                    ok_u = True

        ok_v = False
        for path in nx.all_simple_paths(G, lca, v):
            if len(path) > 0:
                attrs = G.get_edge_data(path[0], path[1])
                if "is_par" in attrs and attrs["is_par"]:
                    ok_v = True

        if ok_u and ok_v:
            H.add_edge(u, v)
            H.add_edge(v, u)

    return H



def read_json_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return json.load(file)


def json_to_graph(json_data):
    task_id = []
    seq_flow_id = []
    par_gateway_id = []
    exc_gateway_id = []
    task_name = []
    name_id_set = {}
    sorted_name = []

    gateways_list = json_data['gateways']
    for gateway in gateways_list:
        gateway_id = gateway['id']
        gateway_type = gateway['type']
        if gateway_type == 'XOR':
            exc_gateway_id.append(gateway_id)
        elif gateway_type == 'AND':
            par_gateway_id.append(gateway_id)

    flows_list = json_data['flows']

    task_list = json_data['tasks']
    for task in task_list:
        id = task['id']
        name = task['label']
        if name != 'ENTRY' and name != 'EXIT':
          name_id_set[name] = id
          task_name.append(name)
    sorted_name = sorted(task_name, key = str.lower)

    for i in range(len(sorted_name)):
      name_key = sorted_name[i]
      id_value = name_id_set.get(name_key)
      task_id.append(id_value)

    G = nx.DiGraph()
    #task的id先存放起来，已知的还有flow列表，其中包含targ和src，网关也只有id，
    #思路：现将task_id添加到graph中，之后遍历边

    for node_id in task_id+par_gateway_id+exc_gateway_id:
        G.add_node(node_id)

    for seq in flows_list:
        src_id = seq["src"]
        tgt_id = seq["tgt"]
        G.add_edge(src_id, tgt_id)

    for node_id in exc_gateway_id:
        for predecessor in G.predecessors(node_id):
          for successor in G.successors(node_id):
              G.add_edge(predecessor, successor)

        G.remove_node(node_id)

    for node_id in par_gateway_id:
        for predecessor in G.predecessors(node_id):
          for successor in G.successors(node_id):
              G.add_edge(predecessor, successor, is_par=true)

        G.remove_node(node_id)


    H = copy.deepcopy(G)

    while not nx.is_directed_acyclic_graph(G):
        cycle = nx.find_cycle(G, orientation="original")
        G.remove_edge(*cycle[-1][:2])

    #通过寻找u，v的最早祖先lca，通过判断lca是否为平行网关，来实现u，v两边的平行
    for (u, v), lca in nx.all_pairs_lowest_common_ancestor(G):
        if u == lca or v == lca:
            continue

        ok_u = False
        for path in nx.all_simple_paths(G, lca, u):
            if len(path) > 0:
                attrs = G.get_edge_data(path[0], path[1])
                if "is_par" in attrs and attrs["is_par"]:
                    ok_u = True

        ok_v = False
        for path in nx.all_simple_paths(G, lca, v):
            if len(path) > 0:
                attrs = G.get_edge_data(path[0], path[1])
                if "is_par" in attrs and attrs["is_par"]:
                    ok_v = True

        if ok_u and ok_v:
            H.add_edge(u, v)
            H.add_edge(v, u)

    for node in task_list:
        node_name = node['label']
        if(node_name=='ENTRY' or node_name == 'EXIT'):
          node_id = node['id']
          H.remove_node(node_id)

    return H


def jsonNameList(json_data):
    task_name = []
    task_list = json_data['tasks']
    for node in task_list:
      node_name = node['label']
      if(node_name == 'ENTRY' or node_name=='EXIT'):
        continue
      if node_name in task_name:
        continue
      task_name.append(node_name)

    task_name = sorted(task_name, key = str.lower)
    return task_name




def sortedNameList(root_dom: Document):
    #传递一个bpmn文件返回他对应的排序好的结点列表
    task = root_dom.getElementsByTagName("task") #注意进行前缀的替换
    #other_tasks = [node for node in root_dom.getElementsByTagName("*") if node.nodeName.endswith("Task")]
    task_nodes = task
    sorted_name_list = []
    for element in task_nodes:
        name = element.getAttribute('name')
        sorted_name_list.append(name)
    sorted_name = sorted(sorted_name_list, key= str.lower)
    return sorted_name #返回每个root_dom中task列表

def twoGraphList(graph1_list,graph2_list):
    finalList = graph1_list+graph2_list
    finalList = list(set(finalList)) #调用内置set函数实现对列表重复元素的删除
    finalList = sorted(finalList, key= str.lower)
    return finalList #返回两个图排序好的name列表


def getEmbedding(graph_list, finalList):
    '''
      对传递进来的图的task进行one-hot编码
    '''
    graphFeature = []
    for i in range(len(finalList)):
      nodeFeature = [0]*(len(finalList))
      graphFeature.append(nodeFeature)
    for name in graph_list:
      if name in finalList:
        index = finalList.index(name)
        graphFeature[index][index] = 1
    graphTensor = torch.tensor(graphFeature)
    return graphTensor


def normalized_adj_matrix(graph1_name_list, final_list, graph1_adj_list):
    list_index = []
    for name in graph1_name_list:
      if name in final_list:
        index = final_list.index(name)
        list_index.append(index)
    list_index.sort()

    graph1_normalized_list=[]
    for i in range(len(final_list)):
      nodeFeature = [0]*(len(final_list))
      graph1_normalized_list.append(nodeFeature)
    for i in range(len(graph1_name_list)):
      x = list_index[i]
      for j in range(len(graph1_name_list)):
        y = list_index[j]
        if(graph1_adj_list[i][j]==1):
          graph1_normalized_list[x][y] = 1
    tensor = torch.Tensor(graph1_normalized_list)
    return tensor

def getCost(name1, name2, adj_matrix1, adj_matrix2):
      # 将两个列表转换为集合
      set1 = set(name1)
      set2 = set(name2)

      # 找到两个集合中的不同元素
      difference = list(set1.symmetric_difference(set2))
      cost = 10*(len(difference))

      for i in range(len(adj_matrix1)):
        for j in range(len(adj_matrix1)):
          if adj_matrix1[i][j] != adj_matrix2[i][j]:
            cost += 1

      return cost

def normalize(A): #加上自连接  A = A+I
    A = torch.eye(A.shape[0]) + A  #返回一个n*n的张量，对角线位置全1，其它位置全0
    d = A.sum(1)  # 所有节点的度，求数组每一行的和
    D = torch.diag(torch.pow(d, -0.5))
    return torch.matmul(torch.matmul(D, A), D)  #D^(-1/2)*A*D^(-1/2)

def expendDimension(X):
    tensor = torch.zeros(50,50)
    zero_list = tensor.numpy().tolist()
    X_list = X.numpy().tolist()
    index = len(X_list[0])
    for i in range(index):
      for j in range(index):
        zero_list[i][j]+= X_list[i][j]
    return torch.tensor(zero_list)

class GCN(nn.Module):  #来增强特征；引入虚拟节点，通过不断循环，获得该图中各个节点的特征。最后将两个图的虚拟节点拿来做对比
    def __init__(self, dim_in, dim_out):
        super(GCN, self).__init__()
        self.fc1 = nn.Linear(dim_in, dim_in, bias=False)  #全连接层 首先初始化一个全连接神经网络
        self.fc2 = nn.Linear(dim_in, dim_in, bias=False)
    def forward(self, X, A):  #X是特征矩阵 A是邻接矩阵 矩阵相乘 X和A相乘(新方法)  A和X相乘(旧方法)
        X_copy = X.clone()  # 复制一份输入Tensor
        X = self.fc1(X.mm(A))
        X = X_copy + X  # 加入残差连接
        X = F.relu(X)
        X_copy = X.clone()  # 复制一份输出Tensor
        X = self.fc2(X.mm(A))
        X = X_copy + X  # 加入残差连接
        X = F.relu(X)
        return X


def euclidean_distance(x, y):
    """This is the squared Euclidean distance."""
    return torch.sqrt( torch.sum(torch.pow(x-y, 2) ) )




def main():
    json_file = '/content/input.struct.json'
    json_data = read_json_file(json_file)
    G_json = json_to_graph(json_data)
    json_adj = np.asarray(nx.adjacency_matrix(G_json).todense())
    json_name = jsonNameList(json_data)


    dom = '/content/struct_model.bpmn'
    root = parse(dom)
    G = bpmn_to_graph(root, prefix = '')
    G_adj = np.asarray(nx.adjacency_matrix(G).todense())
    name = sortedNameList(root)

    nameList = twoGraphList(name, json_name)

    json_embedding = getEmbedding(json_name, nameList)
    bpmn_embedding = getEmbedding(name, nameList)
    json_embedding_amend = expendDimension(json_embedding)
    bpmn_embedding_amend = expendDimension(bpmn_embedding)

    json_normalize = normalized_adj_matrix(json_name, nameList, json_adj)
    json_Laplace = normalize(json_normalize)
    json_Laplace_amend = expendDimension(json_Laplace)

    bpmn_normalize = normalized_adj_matrix(name, nameList, G_adj)
    bpmn_Laplace = normalize(bpmn_normalize)
    bpmn_Laplace_amend = expendDimension(bpmn_Laplace)

    gcn = GCN(50,50)
    f1 = gcn(json_embedding_amend, json_Laplace_amend)
    f2 = gcn(bpmn_embedding_amend, bpmn_Laplace_amend)

    distance = euclidean_distance(f1, f2)
    print(distance)





if __name__ == '__main__':
    main()